# Expected Loss (EL)

## Loss Given Default (LGD) and Exposure at Default (EAD)

### Data Preparation

Import libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
import scipy.stats as stat
import pickle

Load and explore the data

In [2]:
loan_data_preprocessed_backup = pd.read_csv('loan_data_preprocessed.csv')
loan_data_preprocessed = loan_data_preprocessed_backup.copy()
loan_data_preprocessed.shape

/var/folders/hn/11hmxl5x72jg2t7k7m1krds80000gn/T/ipykernel_87841/1878492666.py:1: DtypeWarning: Columns (20,48,56,113,124,125,126,129,130,131,134,140,141,142) have mixed types. Specify dtype option on import or set low_memory=False.
  loan_data_preprocessed_backup = pd.read_csv('loan_data_preprocessed.csv')


(2260668, 279)

In [3]:
loan_data_preprocessed.head(10)

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,addr_state:TX,addr_state:UT,addr_state:VA,addr_state:VT,addr_state:WA,addr_state:WI,addr_state:WV,addr_state:WY,initial_list_status:f,initial_list_status:w
0,0,NaN,NaN,2500,2500,2500.0,36 months,13.56,84.92,C,...,False,False,False,False,False,False,False,False,False,True
1,1,NaN,NaN,30000,30000,30000.0,60 months,18.94,777.23,D,...,False,False,False,False,False,False,False,False,False,True
2,2,NaN,NaN,5000,5000,5000.0,36 months,17.97,180.69,D,...,False,False,False,False,False,False,False,False,False,True
3,3,NaN,NaN,4000,4000,4000.0,36 months,18.94,146.51,D,...,False,False,False,False,True,False,False,False,False,True
4,4,NaN,NaN,30000,30000,30000.0,60 months,16.14,731.78,C,...,False,False,False,False,False,False,False,False,False,True
5,5,NaN,NaN,5550,5550,5550.0,36 months,15.02,192.45,C,...,False,False,False,False,False,False,False,False,False,True
6,6,NaN,NaN,2000,2000,2000.0,36 months,17.97,72.28,D,...,False,False,False,False,False,False,False,False,False,True
7,7,NaN,NaN,6000,6000,6000.0,36 months,13.56,203.79,C,...,False,False,False,False,False,False,False,False,False,True
8,8,NaN,NaN,5000,5000,5000.0,36 months,17.97,180.69,D,...,False,False,False,False,False,False,False,False,False,True
9,9,NaN,NaN,6000,6000,6000.0,36 months,14.47,206.44,C,...,False,False,False,False,False,False,False,False,False,True


Remove empty values from the dataset **loan_data_preprocessed**

In [4]:
loan_data_preprocessed.fillna({'mths_since_last_delinq': 0}, inplace = True)
loan_data_preprocessed.fillna({'mths_since_last_record': 0}, inplace = True)
loan_data_preprocessed.fillna({'dti': 0}, inplace = True)

Select only feactures needed for the LGD and EAD model.

In [5]:
selected_explanatory_variables_lgd_ead = [
    # grade
    'grade:A',
    'grade:B',
    'grade:C',
    'grade:D',
    'grade:E',
    'grade:F',
    'grade:G',
    # home_ownership
    'home_ownership:MORTGAGE',
    'home_ownership:NONE',
    'home_ownership:OTHER',
    'home_ownership:OWN',
    'home_ownership:RENT',
    # verification_status
    'verification_status:Not Verified',
    'verification_status:Source Verified',
    'verification_status:Verified',
    # purpose
    'purpose:car',
    'purpose:credit_card',
    'purpose:debt_consolidation',
    'purpose:educational',
    'purpose:home_improvement',
    'purpose:house',
    'purpose:major_purchase',
    'purpose:medical',
    'purpose:moving',
    'purpose:other',
    'purpose:renewable_energy',
    'purpose:small_business',
    'purpose:vacation',
    'purpose:wedding',
    # initial_list_status
    'initial_list_status:f',
    'initial_list_status:w',
    # term_int
    'term_int',
    # emp_length_int
    'emp_length_int',
    # mths_since_issue_d
    'mths_since_issue_d',
    # mths_since_earliest_cr_line
    'mths_since_earliest_cr_line',
    # funded_amnt
    'funded_amnt',
    # int_rate
    'int_rate',
    # installment
    'installment',
    # annual_inc
    'annual_inc',
    # dti
    'dti',
    # delinq_2yrs
    'delinq_2yrs',
    # inq_last_6mths
    'inq_last_6mths',
    # mths_since_last_delinq
    'mths_since_last_delinq',
    # mths_since_last_record
    'mths_since_last_record',
    # open_acc
    'open_acc',
    # pub_rec
    'pub_rec',
    # total_acc
    'total_acc',
    # acc_now_delinq
    'acc_now_delinq',
    # total_rev_hi_lim
    'total_rev_hi_lim']

In [6]:
reference_category_lgd_ead = [
    'grade:G',
    'home_ownership:RENT',
    'verification_status:Verified',
    'purpose:credit_card',
    'initial_list_status:f']

In [7]:
loan_data_preprocessed_lgd_ead = loan_data_preprocessed.loc[: , selected_explanatory_variables_lgd_ead].copy()
loan_data_preprocessed_lgd_ead = loan_data_preprocessed_lgd_ead.drop(reference_category_lgd_ead, axis = 1)
loan_data_preprocessed_lgd_ead.head(10)

,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,home_ownership:MORTGAGE,home_ownership:NONE,home_ownership:OTHER,home_ownership:OWN,...,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,total_acc,acc_now_delinq,total_rev_hi_lim
0,False,False,True,False,False,False,False,False,False,False,...,18.24,0.0,1.0,0.0,45.0,9.0,1.0,34.0,0.0,42000.0
1,False,False,False,True,False,False,True,False,False,False,...,26.52,0.0,0.0,71.0,75.0,13.0,1.0,44.0,0.0,50800.0
2,False,False,False,True,False,False,True,False,False,False,...,10.51,0.0,0.0,0.0,0.0,8.0,0.0,13.0,0.0,24100.0
3,False,False,False,True,False,False,True,False,False,False,...,16.74,0.0,0.0,0.0,0.0,10.0,0.0,13.0,0.0,7000.0
4,False,False,True,False,False,False,True,False,False,False,...,26.35,0.0,0.0,0.0,0.0,12.0,0.0,26.0,0.0,23100.0
5,False,False,True,False,False,False,True,False,False,False,...,37.94,0.0,3.0,0.0,0.0,18.0,0.0,44.0,0.0,111900.0
6,False,False,False,True,False,False,False,False,False,False,...,2.40,0.0,1.0,0.0,0.0,1.0,0.0,9.0,0.0,0.0
7,False,False,True,False,False,False,False,False,False,False,...,30.10,0.0,0.0,0.0,0.0,19.0,0.0,37.0,0.0,55500.0
8,False,False,False,True,False,False,True,False,False,False,...,21.16,0.0,1.0,32.0,0.0,8.0,0.0,38.0,0.0,22800.0
9,False,False,True,False,False,False,False,False,False,True,...,17.43,1.0,1.0,17.0,0.0,38.0,0.0,58.0,0.0,132500.0


Check the number of features needed for LGD and EAD model against the expected values.

In [8]:
print('Number of features expected: {}'.format(len(selected_explanatory_variables_lgd_ead) - len(reference_category_lgd_ead)))
print('Number of features in dataframe: {}'.format(loan_data_preprocessed_lgd_ead.shape[1]))

Number of features expected: 44
Number of features in dataframe: 44


### Load the Models: LGD and EAD

Retrieve the previously implemented classes **LinearRegression_p_values** and **LogisticRegression_p_values**.

In [9]:
class LinearRegression_p_values:
    def __init__(self):
        self.model = linear_model.LinearRegression(fit_intercept = True, copy_X = True, n_jobs = 1)
    
    def fit(self, X, y, n_jobs = 1):
        self.model.fit(X, y, n_jobs)
        self.coef_ = self.model.coef_
        self.intercept_ = self.model.intercept_

        # calculate p_values for each coefficient
        sse = np.sum((self.model.predict(X) - y) ** 2, axis = 0) / float(X.shape[0] - X.shape[1])
        Xt_dot_X = np.dot(X.T, X)
        Xt_dot_X_float64 = Xt_dot_X.astype(np.float64)
        se = np.array([np.sqrt(np.diagonal(sse * np.linalg.inv(Xt_dot_X_float64)))])
        self.t = self.coef_ / se
        p_values = np.squeeze(2 * (1 - stat.t.cdf(np.abs(self.t), y.shape[0] - X.shape[1])))
        self.p_values = np.append(np.nan, np.round(p_values, 3))

In [10]:
class LogisticRegression_p_values:
    def __init__(self, *args, **kwargs):
        self.model = LogisticRegression(*args, **kwargs)
    
    def fit(self, X, y):
        self.model.fit(X, y)
        self.coef_ = self.model.coef_
        self.intercept_ = self.model.intercept_

        # calculate p_values for each coefficient
        denom = (2.0 * (1.0 + np.cosh(self.model.decision_function(X))))
        denom = np.tile(denom, (X.shape[1], 1)).T
        F_ij = np.dot((X / denom).T, X)
        F_ij_float64 = F_ij.astype(np.float64)
        Cramer_Rao = np.linalg.inv(F_ij_float64)
        sigma_estimates = np.sqrt(np.diagonal(Cramer_Rao))
        z_scores = self.coef_[0] / sigma_estimates
        p_values = [stat.norm.sf(abs(x)) * 2 for x in z_scores]
        self.p_values = np.append(np.nan, np.round(p_values, 3))

Import LGD and EAD models from saved files.

In [11]:
reg_lgd_stage1_model = pickle.load(open('reg_lgd_stage1_model.sav', 'rb'))
reg_lgd_stage2_model = pickle.load(open('reg_lgd_stage2_model.sav', 'rb'))
reg_ead_model = pickle.load(open('reg_ead_model.sav', 'rb'))

### Calculate LGD and EAD

Estimate the Recovery Rate using LGD models from stage 1 and 2.

In [12]:
loan_data_preprocessed['recovery_rate_stage1'] = reg_lgd_stage1_model.model.predict(loan_data_preprocessed_lgd_ead)
loan_data_preprocessed['recovery_rate_stage2'] = reg_lgd_stage2_model.model.predict(loan_data_preprocessed_lgd_ead)
loan_data_preprocessed['recovery_rate'] = loan_data_preprocessed['recovery_rate_stage1'] * loan_data_preprocessed['recovery_rate_stage2']

From previous training and testing process, we need to adjust predicted recovery rate since linear regression model predictions are not bounded between 0 and 1.

In [13]:
loan_data_preprocessed['recovery_rate'] = np.where(loan_data_preprocessed['recovery_rate'] < 0, 0, loan_data_preprocessed['recovery_rate'])
loan_data_preprocessed['recovery_rate'] = np.where(loan_data_preprocessed['recovery_rate'] > 1, 1, loan_data_preprocessed['recovery_rate'])

Calculate the Loss Given Default (LGD)

In [14]:
loan_data_preprocessed['LGD'] = 1 - loan_data_preprocessed['recovery_rate']
loan_data_preprocessed['LGD'].describe()

count    2.260668e+06
mean     9.152343e-01
std      4.925984e-02
min      0.000000e+00
25%      8.800872e-01
50%      8.951782e-01
75%      9.328276e-01
max      1.000000e+00
Name: LGD, dtype: float64

Estimate the Credit Conversion Factor (CCF)

In [15]:
loan_data_preprocessed['CCF'] = reg_ead_model.model.predict(loan_data_preprocessed_lgd_ead)
loan_data_preprocessed['CCF'] = np.where(loan_data_preprocessed['CCF'] < 0, 0, loan_data_preprocessed['CCF'])
loan_data_preprocessed['CCF'] = np.where(loan_data_preprocessed['CCF'] > 1, 1, loan_data_preprocessed['CCF'])

Calculate the Exposure at Default (EAD)

In [16]:
loan_data_preprocessed['EAD'] = loan_data_preprocessed['funded_amnt'] * loan_data_preprocessed['CCF']
loan_data_preprocessed['EAD'].describe()

count    2.260668e+06
mean     1.070386e+04
std      7.245579e+03
min      0.000000e+00
25%      5.052315e+03
50%      8.912392e+03
75%      1.464903e+04
max      4.000000e+04
Name: EAD, dtype: float64

## Probability Default (PD)

### Data Preparation

Import the data from previously preprocessed **loan_data_inputs_train** and **loan_data_inputs_train**

In [17]:
loan_data_inputs_train = pd.read_csv('loan_data_inputs_train.csv')
loan_data_inputs_test = pd.read_csv('loan_data_inputs_test.csv')
loan_data_inputs_pd = pd.concat([loan_data_inputs_train, loan_data_inputs_test], axis = 0)
loan_data_inputs_pd.shape

/var/folders/hn/11hmxl5x72jg2t7k7m1krds80000gn/T/ipykernel_87841/1956906765.py:1: DtypeWarning: Columns (124,125,126,129,130,131,134,140,141,142) have mixed types. Specify dtype option on import or set low_memory=False.
  loan_data_inputs_train = pd.read_csv('loan_data_inputs_train.csv')


(2260668, 389)

In [18]:
loan_data_inputs_pd.head(10)

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,dti:26.4-30.7,dti:30.7-35,dti:>35,mths_since_last_record:Missing,mths_since_last_record:0-15,mths_since_last_record:16-25,mths_since_last_record:26-38,mths_since_last_record:39-87,mths_since_last_record:88-108,mths_since_last_record:>108
0,2064503,NaN,NaN,15000,15000,15000.0,36 months,14.16,513.84,C,...,0,0,0,0,0,0,0,1,0,0
1,1040257,NaN,NaN,11000,11000,11000.0,36 months,8.18,345.62,B,...,0,0,0,1,0,0,0,0,0,0
2,870512,NaN,NaN,10000,10000,10000.0,36 months,11.39,329.24,B,...,0,0,0,1,0,0,0,0,0,0
3,199952,NaN,NaN,10000,10000,10000.0,36 months,10.08,323.05,B,...,0,0,0,0,0,0,0,1,0,0
4,1098582,NaN,NaN,7225,7225,7225.0,36 months,15.61,252.63,D,...,0,0,0,0,0,0,1,0,0,0
5,2194248,NaN,NaN,4200,4200,4200.0,36 months,18.06,151.97,D,...,0,0,0,0,0,0,0,0,1,0
6,1013682,NaN,NaN,35000,35000,35000.0,60 months,18.55,899.28,E,...,0,0,0,1,0,0,0,0,0,0
7,1649058,NaN,NaN,35000,35000,35000.0,60 months,30.74,1148.33,F,...,0,0,0,1,0,0,0,0,0,0
8,1349571,NaN,NaN,7650,7650,7650.0,36 months,12.99,257.73,C,...,0,0,1,1,0,0,0,0,0,0
9,65075,NaN,NaN,4000,4000,4000.0,36 months,16.14,140.91,C,...,0,0,0,1,0,0,0,0,0,0


Since the two dataframes are obtained by **train_test_split** function from scikit-learn library, and the rows in the two dataframes are shuffled, **set_index** function is used to replace the current index with the original row index.

In [19]:
loan_data_inputs_pd = loan_data_inputs_pd.set_index('Unnamed: 0')
loan_data_inputs_pd.head(10)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,dti:26.4-30.7,dti:30.7-35,dti:>35,mths_since_last_record:Missing,mths_since_last_record:0-15,mths_since_last_record:16-25,mths_since_last_record:26-38,mths_since_last_record:39-87,mths_since_last_record:88-108,mths_since_last_record:>108
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
2064503,NaN,NaN,15000,15000,15000.0,36 months,14.16,513.84,C,C2,...,0,0,0,0,0,0,0,1,0,0
1040257,NaN,NaN,11000,11000,11000.0,36 months,8.18,345.62,B,B1,...,0,0,0,1,0,0,0,0,0,0
870512,NaN,NaN,10000,10000,10000.0,36 months,11.39,329.24,B,B3,...,0,0,0,1,0,0,0,0,0,0
199952,NaN,NaN,10000,10000,10000.0,36 months,10.08,323.05,B,B1,...,0,0,0,0,0,0,0,1,0,0
1098582,NaN,NaN,7225,7225,7225.0,36 months,15.61,252.63,D,D1,...,0,0,0,0,0,0,1,0,0,0
2194248,NaN,NaN,4200,4200,4200.0,36 months,18.06,151.97,D,D2,...,0,0,0,0,0,0,0,0,1,0
1013682,NaN,NaN,35000,35000,35000.0,60 months,18.55,899.28,E,E2,...,0,0,0,1,0,0,0,0,0,0
1649058,NaN,NaN,35000,35000,35000.0,60 months,30.74,1148.33,F,F5,...,0,0,0,1,0,0,0,0,0,0
1349571,NaN,NaN,7650,7650,7650.0,36 months,12.99,257.73,C,C2,...,0,0,1,1,0,0,0,0,0,0


Select only the required features for the PD model.

In [20]:
selected_explanatory_variables_pd = [
    # grade
    'grade:A',
    'grade:B',
    'grade:C',
    'grade:D',
    'grade:E',
    'grade:F',
    'grade:G',
    # home_ownership
    'home_ownership:RENT_OTHER_NONE_ANY',
    'home_ownership:OWN',
    'home_ownership:MORTGAGE',
    # addr_state
    'addr_state:IA_AL_LA_OK_AR_MS_NV',
    'addr_state:NM_HI_SD',
    'addr_state:NY',
    'addr_state:FL',
    'addr_state:MO_MD_NC_TN_NJ_IN',
    'addr_state:CA',
    'addr_state:PA_MI_KY',
    'addr_state:VA_AK_MN_OH_AZ',
    'addr_state:TX',
    'addr_state:NE_DE',
    'addr_state:MA_WI_UT',
    'addr_state:GA_WY_RI_IL_MT',
    'addr_state:CT_WA_KS_ND_CO_SC',
    'addr_state:WV_NH_DC_OR_VT_ID_ME',
    # verification_status
    'verification_status:Not Verified',
    'verification_status:Source Verified',
    'verification_status:Verified',
    # purpose
    'purpose:edu_smbus_rnenergy_mov',
    'purpose:debt_consolidation',
    'purpose:med_oth_wed_vac_hou_mjpur_hmimpr',
    'purpose:credit_card',
    'purpose:car',
    # initial_list_status
    'initial_list_status:f',
    'initial_list_status:w',
    # term
    'term:36',
    'term:60',
    # emp_length
    'emp_length:0',
    'emp_length:1',
    'emp_length:2-6',
    'emp_length:7',
    'emp_length:8-9',
    'emp_length:10',
    # mths_since_issue_d
    'mths_since_issue_d:<2',
    'mths_since_issue_d:3-5',
    'mths_since_issue_d:6-7',
    'mths_since_issue_d:8-15',
    'mths_since_issue_d:16-45',
    'mths_since_issue_d:46-65',
    'mths_since_issue_d:>65',
    # int_rate
    'int_rate:<10.446',
    'int_rate:10.446-13.014',
    'int_rate:13.014-17.123',
    'int_rate:17.123-22.772',
    'int_rate:>22.772',
    # mths_since_earliest_cr_line
    'mths_since_earliest_cr_line:<142',
    'mths_since_earliest_cr_line:142-162',
    'mths_since_earliest_cr_line:163-244',
    'mths_since_earliest_cr_line:245-284',
    'mths_since_earliest_cr_line:285-345',
    # inq_last_6mths
    'inq_last_6mths:0',
    'inq_last_6mths:1-3',
    'inq_last_6mths:>3',
    # open_acc
    'open_acc:<4',
    'open_acc:4-9',
    'open_acc:10-22',
    'open_acc:23-34',
    'open_acc:>34',
    # total_rev_hi_lim
    'total_rev_hi_lim:<5K',
    'total_rev_hi_lim:5K-10K',
    'total_rev_hi_lim:10K-20K',
    'total_rev_hi_lim:20K-30K',
    'total_rev_hi_lim:30K-40K',
    'total_rev_hi_lim:40K-50K',
    'total_rev_hi_lim:50K-100K',
    'total_rev_hi_lim:>=100K',
    # annual_inc
    'annual_inc:<20K',
    'annual_inc:20K-30K',
    'annual_inc:30K-40K',
    'annual_inc:40K-50K',
    'annual_inc:50K-60K',
    'annual_inc:60K-70K',
    'annual_inc:70K-80K',
    'annual_inc:80K-90K',
    'annual_inc:90K-100K',
    'annual_inc:100K-120K',
    'annual_inc:120K-140K',
    'annual_inc:>=140K',
    # dti
    'dti:<1.9',
    'dti:1.9-4.8',
    'dti:4.8-6.9',
    'dti:6.9-10.5',
    'dti:10.5-12.7',
    'dti:12.7-15.6',
    'dti:15.6-19.2',
    'dti:19.2-24.2',
    'dti:24.2-26.4',
    'dti:26.4-30.7',
    'dti:30.7-35',
    'dti:>35',
    # mths_since_last_delinq
    'mths_since_last_delinq:Missing',
    'mths_since_last_delinq:0-18',
    'mths_since_last_delinq:19-67',
    'mths_since_last_delinq:>=68',
    # mths_since_last_record
    'mths_since_last_record:Missing',
    'mths_since_last_record:0-15',
    'mths_since_last_record:16-25',
    'mths_since_last_record:26-38',
    'mths_since_last_record:39-87',
    'mths_since_last_record:88-108',
    'mths_since_last_record:>108']

In [21]:
reference_category_pd = [
    'grade:G',
    'home_ownership:RENT_OTHER_NONE_ANY',
    'addr_state:IA_AL_LA_OK_AR_MS_NV',
    'verification_status:Verified',
    'purpose:edu_smbus_rnenergy_mov',
    'initial_list_status:f',
    'term:60',
    'emp_length:0',
    'mths_since_issue_d:>65',
    'int_rate:>22.772',
    'mths_since_earliest_cr_line:<142',
    'inq_last_6mths:>3',
    'open_acc:<4',
    'total_rev_hi_lim:<5K',
    'annual_inc:<20K',
    'dti:>35',
    'mths_since_last_delinq:0-18',
    'mths_since_last_record:0-15']

In [22]:
loan_data_inputs_pd_temp = loan_data_inputs_pd.loc[: , selected_explanatory_variables_pd].copy()
loan_data_inputs_pd_temp = loan_data_inputs_pd_temp.drop(reference_category_pd, axis = 1)
loan_data_inputs_pd_temp.shape

(2260668, 92)

In [23]:
loan_data_inputs_pd_temp.head(10)

,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,home_ownership:OWN,home_ownership:MORTGAGE,addr_state:NM_HI_SD,addr_state:NY,...,dti:30.7-35,mths_since_last_delinq:Missing,mths_since_last_delinq:19-67,mths_since_last_delinq:>=68,mths_since_last_record:Missing,mths_since_last_record:16-25,mths_since_last_record:26-38,mths_since_last_record:39-87,mths_since_last_record:88-108,mths_since_last_record:>108
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
2064503,False,False,True,False,False,False,False,False,0,False,...,0,1,1,0,0,0,0,1,0,0
1040257,False,True,False,False,False,False,False,True,0,False,...,0,0,1,0,1,0,0,0,0,0
870512,False,True,False,False,False,False,False,False,0,False,...,0,0,1,0,1,0,0,0,0,0
199952,False,True,False,False,False,False,False,True,0,False,...,0,0,1,0,0,0,0,1,0,0
1098582,False,False,False,True,False,False,False,False,0,False,...,0,0,1,0,0,0,1,0,0,0
2194248,False,False,False,True,False,False,False,True,1,False,...,0,0,1,0,0,0,0,0,1,0
1013682,False,False,False,False,True,False,False,True,0,False,...,0,0,1,0,1,0,0,0,0,0
1649058,False,False,False,False,False,True,True,False,0,False,...,0,0,1,0,1,0,0,0,0,0
1349571,False,False,True,False,False,False,False,False,0,False,...,0,1,1,0,1,0,0,0,0,0


### Load the PD Model

Import the PD model from previous notebook output.

In [24]:
reg_pd_model = pickle.load(open('reg_pd_model.sav', 'rb'))

### Calculate PD

In [25]:
loan_data_inputs_pd['PD'] = reg_pd_model.model.predict_proba(loan_data_inputs_pd_temp)[ : ][ : , 0]
loan_data_inputs_pd.head(10)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,dti:30.7-35,dti:>35,mths_since_last_record:Missing,mths_since_last_record:0-15,mths_since_last_record:16-25,mths_since_last_record:26-38,mths_since_last_record:39-87,mths_since_last_record:88-108,mths_since_last_record:>108,PD
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
2064503,NaN,NaN,15000,15000,15000.0,36 months,14.16,513.84,C,C2,...,0,0,0,0,0,0,1,0,0,0.184779
1040257,NaN,NaN,11000,11000,11000.0,36 months,8.18,345.62,B,B1,...,0,0,1,0,0,0,0,0,0,0.085229
870512,NaN,NaN,10000,10000,10000.0,36 months,11.39,329.24,B,B3,...,0,0,1,0,0,0,0,0,0,0.099695
199952,NaN,NaN,10000,10000,10000.0,36 months,10.08,323.05,B,B1,...,0,0,0,0,0,0,1,0,0,0.013646
1098582,NaN,NaN,7225,7225,7225.0,36 months,15.61,252.63,D,D1,...,0,0,0,0,0,1,0,0,0,0.270125
2194248,NaN,NaN,4200,4200,4200.0,36 months,18.06,151.97,D,D2,...,0,0,0,0,0,0,0,1,0,0.049696
1013682,NaN,NaN,35000,35000,35000.0,60 months,18.55,899.28,E,E2,...,0,0,1,0,0,0,0,0,0,0.264314
1649058,NaN,NaN,35000,35000,35000.0,60 months,30.74,1148.33,F,F5,...,0,0,1,0,0,0,0,0,0,0.246184
1349571,NaN,NaN,7650,7650,7650.0,36 months,12.99,257.73,C,C2,...,0,1,1,0,0,0,0,0,0,0.166188


In [26]:
loan_data_inputs_pd['PD'].describe()

count    2.260668e+06
mean     1.256591e-01
std      1.003750e-01
min      3.481111e-03
25%      4.516904e-02
50%      1.004267e-01
75%      1.818559e-01
max      6.819399e-01
Name: PD, dtype: float64

## Expected Loss (EL)

### Calculate the Expected Loss (EL)

Concatenate the two dataframes **loan_data_preprocessed** (for LGD and EAD model) and **loan_data_inputs_pd** (for PD model).

In [27]:
loan_data_preprocessed_all = pd.concat([loan_data_preprocessed, loan_data_inputs_pd], axis = 1)
loan_data_preprocessed_all.shape

(2260668, 674)

In [28]:
loan_data_preprocessed_all.head(10)

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,dti:30.7-35,dti:>35,mths_since_last_record:Missing,mths_since_last_record:0-15,mths_since_last_record:16-25,mths_since_last_record:26-38,mths_since_last_record:39-87,mths_since_last_record:88-108,mths_since_last_record:>108,PD
0,0,NaN,NaN,2500,2500,2500.0,36 months,13.56,84.92,C,...,0,0,0,0,0,0,1,0,0,0.045120
1,1,NaN,NaN,30000,30000,30000.0,60 months,18.94,777.23,D,...,0,0,0,0,0,0,1,0,0,0.072039
2,2,NaN,NaN,5000,5000,5000.0,36 months,17.97,180.69,D,...,0,0,1,0,0,0,0,0,0,0.044777
3,3,NaN,NaN,4000,4000,4000.0,36 months,18.94,146.51,D,...,0,0,1,0,0,0,0,0,0,0.040799
4,4,NaN,NaN,30000,30000,30000.0,60 months,16.14,731.78,C,...,0,0,1,0,0,0,0,0,0,0.044457
5,5,NaN,NaN,5550,5550,5550.0,36 months,15.02,192.45,C,...,0,1,1,0,0,0,0,0,0,0.027210
6,6,NaN,NaN,2000,2000,2000.0,36 months,17.97,72.28,D,...,0,0,1,0,0,0,0,0,0,0.059485
7,7,NaN,NaN,6000,6000,6000.0,36 months,13.56,203.79,C,...,0,0,1,0,0,0,0,0,0,0.048122
8,8,NaN,NaN,5000,5000,5000.0,36 months,17.97,180.69,D,...,0,0,1,0,0,0,0,0,0,0.065594
9,9,NaN,NaN,6000,6000,6000.0,36 months,14.47,206.44,C,...,0,0,1,0,0,0,0,0,0,0.034800


Calculate the Expected Loss (EL = PD x LGD x EAD).

In [29]:
loan_data_preprocessed_all['EL'] = loan_data_preprocessed_all['PD'] * loan_data_preprocessed_all['LGD'] * loan_data_preprocessed_all['EAD']
loan_data_preprocessed_all['EL'].describe()

count    2.260668e+06
mean     1.241052e+03
std      1.497292e+03
min      0.000000e+00
25%      2.884330e+02
50%      6.834339e+02
75%      1.597053e+03
max      1.722866e+04
Name: EL, dtype: float64

In [30]:
loan_data_preprocessed_all[['funded_amnt', 'PD', 'LGD', 'EAD', 'EL']].head(10)

,funded_amnt,funded_amnt,PD,LGD,EAD,EL
0,2500,2500,0.045120,1.000000,1861.548279,83.992450
1,30000,30000,0.072039,1.000000,25998.301801,1872.885946
2,5000,5000,0.044777,0.865238,3828.575744,148.329722
3,4000,4000,0.040799,0.866612,3032.776900,107.228921
4,30000,30000,0.044457,1.000000,26372.284028,1172.435573
5,5550,5550,0.027210,1.000000,4304.069830,117.111593
6,2000,2000,0.059485,0.868165,1564.412761,80.790664
7,6000,6000,0.048122,1.000000,4200.089654,202.115260
8,5000,5000,0.065594,0.851322,3847.710639,214.863124
9,6000,6000,0.034800,1.000000,4445.479837,154.702556


In [31]:
print(loan_data_preprocessed_all['EL'].sum() / loan_data_preprocessed_all['funded_amnt'].sum())

funded_amnt    0.082508
funded_amnt    0.082508
dtype: float64


The observed EL values are between 2% and 10% of the capital.